In [1]:
from jamo import h2j, j2h, get_jamo_class, is_jamo

In [2]:
sents = ['향상도 값은 불순도 값과 반대로 클 수록 좋다.', '본 포스팅을 통해 일정액의 수수료를 제공받을 수 있음', '열여춘향 슈졀가라  라 완산판본 춘향전 (57/168) *(요지) : 곤륜산에 있는 선인이 살았다는 못.', '9일 한국부동산원에 따르면 이달 5일 기준 서울 아파트 매매수급지수는 지난주(101.0)보다 4.9포인트 내린 96.1을 기록했다.', '요새 이것 때문에 공부하느라 정신이 없었는데... 이상하게 맘이 급할땐 항상 일이 몰리더라구요.', '베란다 욕실 욕실에 창문이 있고 깔끔하네요 ....', '생김새 4~6월 노란색 꽃이 잎겨드랑이에서 나온 긴 자루에 달려 1개씩 핀다.', '*(일난초) : 아침나절 햇살이 퍼져 따뜻해짐.', '장군방()은 중앙정부()의 장군()이 일보는 곳임.', '둘러보니 매장바로 오른쪽에 긴 테이블도 구비되어있더라구요', '찬물에 10분 콜드브루처럼 우려낸 다음 3분 더 끓이시면 맛있는 육수가 완성된답니다 ~', '오비완 케노비 피규어는 2020년 제품인 무스타파의 대결 (75269)에 포함된 피규어로 제가 처음으로 갖게 된 영화 버전의 오비완 피규어입니다.']
decomposed = [h2j(x) for x in sents]

In [3]:
decomposed[0]

'향상도 값은 불순도 값과 반대로 클 수록 좋다.'

In [4]:
from tokenizers import SentencePieceBPETokenizer, normalizers
from tokenizers.processors import TemplateProcessing

tok = SentencePieceBPETokenizer()
tok.normalizer = normalizers.NFKD()
tok.post_processor = TemplateProcessing(
            single="[CLS] $A",
            special_tokens=[("[CLS]", 4)],
        )

tok.enable_padding(direction='right', pad_id=0, pad_token='[PAD]', length=128)
tok.enable_truncation(128, stride=0, strategy='longest_first')

In [5]:
tok.train_from_iterator(
            iterator=decomposed,
            vocab_size=100,
            min_frequency=1,  # 단어의 최소 발생 빈도, 5
            limit_alphabet=100,
            show_progress=True,
            special_tokens=["[PAD]", "[SOS]", "[EOS]", "[UNK]", "[CLS]", "[SEP]"],
        )

In [6]:
res = tok.encode_batch(sents, add_special_tokens=True)

In [7]:
res[0].tokens

['[CLS]',
 '▁',
 'ᄒ',
 'ᅣ',
 'ᆼ',
 'ᄉ',
 'ᅡᆼ',
 '도',
 '▁ᄀ',
 'ᅡ',
 'ᆹ',
 'ᄋ',
 'ᅳᆫ',
 '▁',
 '부',
 'ᆯ',
 'ᄉ',
 'ᅮᆫ',
 '도',
 '▁ᄀ',
 'ᅡ',
 'ᆹ',
 'ᄀ',
 'ᅪ',
 '▁',
 'ᄇ',
 'ᅡᆫ',
 'ᄃ',
 'ᅢ',
 '로',
 '▁',
 'ᄏ',
 'ᅳ',
 'ᆯ',
 '▁ᄉ',
 'ᅮ',
 '로',
 'ᆨ',
 '▁ᄌ',
 'ᅩ',
 'ᇂ',
 '다',
 '.',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[P

In [8]:
encoded = [x.ids for x in res]

decoded = tok.decode_batch(encoded, skip_special_tokens=True)

In [9]:
from utils.utils import reconstruct_jamo

In [10]:
[reconstruct_jamo(s) for s in decoded] == sents

True

In [11]:
tok.save('asdf.tok')

In [12]:
from tokenizers import Tokenizer
_tok = Tokenizer.from_file('asdf.tok')

In [13]:
tok.encode(decomposed[-1]).tokens == _tok.encode(decomposed[-1]).tokens

True

In [14]:
x = tok.encode(decomposed[-1]).ids

In [15]:
x = [_tok.get_vocab()['[SOS]']] + x + [_tok.get_vocab()['[EOS]']]

In [16]:
tok.decode(x, skip_special_tokens=True)

'오비완 케노비 피규어는 2020년 제품인 무스타파의 대결 (75269)에 포함된 피규어로 제가 처음으로 갖게 된 영화 버전의 오비완 피규어입니다.'

In [17]:
for x in tok.encode_batch(decomposed):
    print(x.attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [26]:
set(tok.get_vocab().values()) == set(range(100))



True

In [1]:
from nltk.translate.gleu_score import corpus_gleu

In [23]:
corpus_gleu([['안녕하세요.']], ['녕하세요.']) * 100

77.77777777777779